In [2]:
import io
import os
import numpy as np
import glob
from IPython.display import display

from six import BytesIO

from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
import cv2

In [5]:
def load_models():
  model_detection = tf.saved_model.load('model_detection')
  model_classification = tf.keras.models.load_model('model_classification')
  return model_detection, model_classification

In [7]:
model_detection, model_classification = load_models()

dients. Will likely fail if a gradient is requested.


In [6]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using tf.convert_to_tensor.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with tf.newaxis.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

In [167]:
classes = { 0:'Vitesse limite 20km h',
            1:'Vitesse limite 30km h', 
            2:'Vitesse limite 50km h', 
            3:'Vitesse limite 60km h', 
            4:'Vitesse limite 70km h', 
            5:'Vitesse limite 80km h', 
            6:'Fin de limite de vitesse 80kmh', 
            7:'Vitesse limite 100km h', 
            8:'Vitesse limite 120km h', 
            9:'Depassement interdit sauf 2 roues', 
            10:'Dépassement interdit pour plus de 3.5t', 
            11:'Priorite a droite a l intersection', 
            12:'Route prioritaire', 
            13:'Cedez le passage', 
            14:'Stop', 
            15:'Interdit aux vehicules', 
            16:'Vehicules de plus de 3.5t interdits', 
            17:'Sens interdit', 
            18:'Panneau danger', 
            19:'Virage dangereux a gauche', 
            20:'Virage dangereux a droite', 
            21:'Succession de virages', 
            22:'Cassis ou dos d anes', 
            23:'Route glissante', 
            24:'Chaussee retrecie par la droite', 
            25:'Travaux', 
            26:'Feux tricolore', 
            27:'Passage pieton', 
            28:'Attention enfants', 
            29:'Attention velos', 
            30:'Attention neige verglas',
            31:'Passage animaux sauvages', 
            32:'Fin de toutes interdictions', 
            33:'Direction obligatoire a droite', 
            34:'Direction obligatoire a gauche', 
            35:'Direction obligatoire tout droit', 
            36:'Direction obligatoire tout droit ou a droite', 
            37:'Direction obligatoire tout droit ou gauche', 
            38:'Contournement obligatoire par la droite', 
            39:'Contournement obligatoire par la gauche', 
            40:'Giratoire', 
            41:'Fin de l interdiction de depassement', 
            42:'Fin de l interdiction de depassement plus de 3.5t' }

In [174]:
def create_boxes(image, model, output_dict, classes, min_score=0.3, line_thickness=3, prediction=True):
  boxes = output_dict['detection_boxes']
  scores = output_dict['detection_scores']
  scores = np.array(scores)
  # Get the index of the boxes where the score are at a certain level
  indices = np.where(scores > min_score)

  # For each index 
  for indice in indices[0]:
    # Get all data coordinates of the box
    xmin = int(1360*boxes[indice][1])
    xmax = int(1360*boxes[indice][3])
    ymin = int(800*boxes[indice][0])
    ymax = int(800*boxes[indice][2])

    # Crop the image to have only the sign and resize it to 30x30 for the classifier
    image_crop = image[ymin:ymax, xmin:xmax]
    image_crop = cv2.resize(image_crop, (30, 30))

    # Draw a rectangle around the sign
    image = cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), line_thickness)
    image_crop = image_crop.reshape(1, 30, 30, 3)

    
    if prediction == True:
      # Predict the proba of each sign 
      pred = model.predict_proba(image_crop)
      
      # Display the label predicted only if score is above 0.8
      if np.max(pred) > 0.8:
        pred_text = classes[np.argmax(pred)]

        # Display the label next to the box
        if xmax + len(pred_text) *10 > 1360:
          image = cv2.rectangle(image, (xmin - len(pred_text) *10, ymax - 20), (xmin, ymax), (255,0,0), -1)
          image = cv2.putText(image, pred_text, (xmin - len(pred_text) *10, ymax - 5), 
          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
        else:
          image = cv2.rectangle(image, (xmax, ymax), (xmax + len(pred_text) *10, ymax - 20), (255,0,0), -1)
          image = cv2.putText(image, pred_text, (xmax, ymax - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
  
  return image

In [136]:
def process_image(image_path, min_score=0.25, prediction=True):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (1360, 800))

    output_dict = run_inference_for_single_image(model_detection, image)
        
    image_boxes = create_boxes(image,
    model_classification, output_dict, classes, min_score=min_score, prediction=prediction)

    return image_boxes

In [1]:
for i, image_path in enumerate(
      glob.glob('tensorflow_api/models/research/object_detection/images/images/*.jpeg')):

      if i < 50:
            continue

      if i > 51:
            break
      
      process_image(image_path, min_score=0.25, prediction=True)


NameError: name 'glob' is not defined

In [147]:
def process_video(path, model_detection, model_classification, record_name=None):
    cap = cv2.VideoCapture(path)
    if record_name:
        result = cv2.VideoWriter(record_name,  cv2.VideoWriter_fourcc(*'MJPG'), 15, (1360, 800)) 

    while True:
        ret, image = cap.read()
        if ret == True:
            output_dict = run_inference_for_single_image(model_detection, image)

            image = create_boxes(cv2.resize(image, (1360, 800)), 
            model_classification, output_dict, classes, min_score=0.3, prediction=True)
                        
            cv2.imshow('detect', image)
            result.write(image)  
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    result.release()
    cv2.destroyAllWindows() 

process_video('test11.mp4', model_detection, model_classification, 'output11.avi')